In [1]:
import numpy as np
from paik.solver import Solver, DEFAULT_SOLVER_PARAM_M7_NORM

solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [2]:
num_im = 100
num_ta = 100
num_sols = 100

solver.shrink_ratio = 0.0
J_im, P_im, F_im = solver.random_sample_JPF(num_samples=num_im)
J_ta, P_ta, F_ta = solver.random_sample_JPF(num_samples=num_ta)

In [3]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=1).fit(np.column_stack((solver._J_tr, solver._P_tr)))

In [4]:
def get_F(J, P):
    J, P = np.atleast_2d(J), np.atleast_2d(P)
    _, idx = knn.kneighbors(np.column_stack((J, P)))
    return solver._F[idx.flatten()]

In [5]:
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, (j_im, f_im) in enumerate(zip(J_im, F_im)):
    # F = np.full_like(F_ta, *f_im)
    j_im = np.tile(j_im, (num_ta, 1))
    F = get_F(j_im, P_ta)
    J = solver.solve(P=P_ta, F=F, num_sols=num_sols, return_numpy=True)
    l2_errs[i], ang_errs[i] = solver.evaluate_solutions(J, P_ta, return_col=True)
    config_errs[i] = np.linalg.norm(J - j_im, axis=2).mean(axis=0)

In [6]:
import pandas as pd

df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": ang_errs.flatten(),
        "config_err": config_errs.flatten(),
    }
)

df.describe()

,l2_err,ang_err,config_err
count,10000.000000,10000.000000,10000.000000
mean,0.005232,0.062381,4.290809
std,0.012598,0.200133,1.332007
min,0.000149,0.000977,1.004540
25%,0.002012,0.012919,3.299007
50%,0.003167,0.022440,4.181115
75%,0.004821,0.043926,5.188488
max,0.437138,3.127906,9.074404


In [7]:
import numpy as np
import pandas as pd
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

# -> unrefined solutions
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, p in enumerate(P_ta):
    (
        solutions,
        l2_errors,
        angular_errors,
        _,
        _,
        _,
    ) = ik_solver.solve(
        p, n=num_im, refine_solutions=False, return_detailed=True
    )  # type: ignore

    solutions = solutions.detach().cpu().numpy()

    l2_errs[:, i] = l2_errors
    ang_errs[:, i] = angular_errors

    for j, j_im in enumerate(J_im):
        config_errs[j, i] = np.linalg.norm(solutions - j_im, axis=1).mean()

config.py: Using device 'cuda:0'
set_seed() - random int:  541
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual

In [8]:
df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": ang_errs.flatten(),
        "config_err": config_errs.flatten(),
    }
)

df.describe()

,l2_err,ang_err,config_err
count,10000.000000,10000.000000,10000.000000
mean,0.008542,0.043220,5.109644
std,0.022180,0.161819,0.778597
min,0.000106,0.000488,2.582034
25%,0.002389,0.008652,4.555377
50%,0.003756,0.013532,5.072735
75%,0.006233,0.024351,5.644392
max,0.471460,3.080453,7.747784
